In [1]:
pip install pandasql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import wrds
import pandas
import numpy
import datetime
import pandasql
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

In [3]:
conn = wrds.Connection()

Enter your WRDS username [anonymous]:johnbilsel
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: n
You can create this file yourself at any time
with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:
# get MSF from WRDS
msf = conn.raw_sql("""select a.permno, a.date, a.permco, a.prc, a.ret, a.shrout,
                  b.shrcd, b.exchcd
                  from crsp.msf a 
                  inner join crsp.msenames b on a.permno=b.permno and b.namedt<=a.date and a.date<=b.nameendt
                  where a.date>='01/01/1965'""",
                  date_cols=['date'])

# align dates to end of month
msf['date'] = msf['date'] + MonthEnd(0)
msf = msf.loc[(msf['shrcd'].isin([10,11])) & (msf['exchcd'].isin([1,2,3]))]

In [5]:
crsp = msf.sort_values(['permno','date'])
crsp = crsp.set_index('date').groupby('permno')['ret'].resample('M').sum(min_count=1).reset_index()

In [6]:
# define function to estimate regression
def regress(df, yvar, xvars):
    y = df[yvar]
    x = df[xvars]
    x['intercept'] = 1
    result = sm.OLS(y, x).fit()
    return result.params

In [7]:
# look back up to 240 months back
num_lags = 240

# container for dataframes generated in each round of the loop below
results_list = []
crsp2 = crsp.copy()

# create a variable LAGRET in the CRSP dataframe
# increment the lag in each round of the loop
# LAGRET is the RET of the stock from NLAGS months ago
# notice that we create LAGRET in a stepwise fashion for memory efficiency
# estimate univariate regression in each round of the loop and store parameters in the container RESULTS_LIST
for lag in range(1, num_lags+1):
    crsp2['lagret'] = crsp2.groupby('permno')['ret'].shift(lag)
    results = (crsp2.loc[~crsp2.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]
               .groupby('date').apply(regress, 'ret', ['lagret']))
    results['nlag'] = lag
    results_list.append(results)

# stack dataframes from RESULTS_LIST vertically in COEFFS
coeffs = pandas.concat(results_list)

KeyboardInterrupt: 

In [ ]:
# compute mean coefficient estimates from each NLAG model across DATES
coeffs_m = coeffs.groupby('nlag')['lagret'].mean().to_frame('coeff').reset_index()

# compute t-stats
coeffs_t = (coeffs.groupby('nlag')
            .apply(lambda df: stats.ttest_1samp(df['lagret'], 0)[0].round(2))
            .to_frame('t')
            .reset_index())

# merge mean coefficient estimates and their t-stats
coeffs_data = pandas.merge(coeffs_m,
                           coeffs_t,
                           how='inner', on='nlag')

In [ ]:
# plot coefficients from univariate regressions of RET on LAGRET
coeffs_plot = coeffs_data.plot(x='nlag', y='coeff', figsize=(15,5))

# set Y-axis to be bounded from bottom at -0.02 to make plot look better
plt.gca().set_ylim(bottom=-0.02)

# draw vertical lines at multiples of 12-months
[coeffs_plot.axvline(x, color='red', linestyle='--') for x in range(240+1) if x%12==0 and x>0]

In [ ]:
# make a copy of CRSP
crsp3 = crsp.copy()

# get returns at lags 1-12, and lags at multiple of 12 thereafter.
for lag in range(1, num_lags+1):
    if lag<=12 or (lag>12 and lag%12==0):
        crsp3['lag'+str(lag)] = crsp3.groupby('permno')['ret'].shift(lag)

In [ ]:
# container for dataframes generated in each round of the loop below
results_list = []

# loop over 3 model specifications in HS2008 Table 1 Panel B
# transpose parameter estimates from wide to long
for lagmax in (36, 120, 240):
    df = crsp3.loc[:, 'permno':'lag'+str(lagmax)]
    df = df.loc[~df.isin([numpy.nan, numpy.inf, -numpy.inf]).any(1)]
    x = df.columns[df.columns.str.startswith('lag')]
    results = df.groupby('date').apply(regress, 'ret', [*x]).reset_index()
    results['model'] = lagmax
    results = pandas.melt(results, id_vars=['date','model'], value_vars=[*x], value_name='coeff')
    results_list.append(results)
    
# stack dataframes from RESULTS_LIST vertically in COEFFS_MULT 
coeffs_mult = pandas.concat(results_list)

In [ ]:
# compute mean coefficient estimates from each MODEL across DATES
coeffs_m = coeffs_mult.groupby(['model','variable'])['coeff'].mean().to_frame('coeff').reset_index()

# compute t-stats
coeffs_t = (coeffs_mult.groupby(['model','variable'])
            .apply(lambda df: stats.ttest_1samp(df['coeff'], 0)[0].round(2))
            .to_frame('t')
            .reset_index())

# merge mean coefficient estimates and their t-stats
coeffs_data = pandas.merge(coeffs_m,
                           coeffs_t,
                           how='inner', on=['model','variable'])

# keep the numeric portion of the VARIABLE column
coeffs_data['variable'] = coeffs_data['variable'].str.split('lag').str[1].astype(int)

# sort on MODEL and VARIABLE
coeffs_data = coeffs_data.sort_values(['model','variable'])